In [1]:
import numpy as np
import torch
from NeuralNet import NeuralNetwork as Network
import wandb

In [3]:
# Model hyperparameters
Filters = 256
Layers = 40
HistoryDepth = 8
TargetDevice = "cuda"
model = Network(Filters, HistoryDepth + 1, Layers, kernal_size=3)

# Model source
modelPath = "../../Models/HumanModels/256f40l/5.pt"


model.load_state_dict(torch.load(modelPath))
model.to(TargetDevice)
model.eval()

example = np.zeros((1, HistoryDepth + 1, 15, 15), dtype=np.float32) 
example = torch.tensor(example).to(TargetDevice)

traced_script_module = torch.jit.trace(model, example)
traced_script_module.save(f'../../Models/HumanModels/scripted/{Filters}f_{Layers}l_{HistoryDepth}hd.pt')